In [4]:
# Import packages
import mne
import scipy
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import fooof
from scipy.io import loadmat, savemat
import pandas as pd

# Custom functions
from nvr_eeg_power_functions_new import castToList, eeg_pow_extract, NVR_eeg_features

# EEG feature extraction
### Extract from EEG data the delta, theta, alpha, beta and gamma frequency band power time-series.
### Create a dataframe for LMM analysis in R.
#### 2024 by Antonin Fourcade

In [5]:
# Define frequency bands of interest
# delta (δ; 0–4 Hz), theta (θ; 4–8 Hz), alpha (α; 8–12 Hz), beta (β; 12–ƒ30 Hz) and gamma (γ; 30–45 Hz).
bands = fooof.bands.Bands({'delta' : [0.3, 4],
                           'theta' : [4, 8],
                           'alpha' : [8, 13], 
                           'beta' : [13, 30],
                           'gamma': [30, 45]})

# Define ROI
roi = ['Pz', 'P3', 'P4', 'P7', 'P8', 'O1', 'O2', 'Oz']

# Path to EEG montage
mon_path = 'E:/NeVRo/Nevro_montage.loc'

# Parameters
m_conds = ['mov', 'nomov'] # head movements conditions
a_conds = ['LA', 'HA'] # emotional arousal conditions
c_styles = 'SBA' # rollercoaster sequence: Space-Break-Andes

# Parameters for filtering, TFR and power integration
fs = 250 # EEG sampling frequency (Hz)
hpf_freq = 0.3 # high-pass filter frequency (Hz)
lpf_freq = 45 # low-pass filter frequency (Hz)
len_win = 2  # 2s window
overlap = 0.5  # 50% overlap
freqs = np.arange(0.5, 125.5, 0.5)  # resolution 0.5Hz
times = np.arange(1, 270, 1)  # resolution 1s

# length of the data in seconds to mirror at the beginning and at the end of the data
mirror_len=80
# length of the data in seconds to mirror at the beginning and at the end of the break
mirror_break=30
# length of the mirrored data in seconds to keep at the beginning and at the end of the data 
cut=35

# Paths to the data and for saving
path_data = 'E:/NeVRo/Data/'
path_eeg = path_data + '13_eeglab2mne/'
path_fbp = path_data + 'Frequency_Bands_Power/'
path_pwr = path_fbp + 'EEG_pow/'
path_pwr_roi = path_fbp + 'ROI_EEG_pow/'
path_ar = path_data + 'ratings/class_bins/'

# Initialize the dictionary to store the data
eeg_features = {
    'mov': [],
    'nomov': []
}

# All mov are included in nomov, but nomov have more SJs
SJs_nomov = [2, 4, 5, 6, 7, 8, 9, 11, 13, 14, 15, 17, 18, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 34, 36, 37, 39, 44]
SJs_mov = [2, 4, 5, 6, 8, 9, 11, 13, 14, 17, 18, 20, 21, 22, 24, 25, 27, 28, 29, 31, 34, 36, 37, 39, 44]

# Get the eeg_features parameters for each SJ and each mov condition

In [6]:
# BE CAREFUL IT TAKES A LONG TIME (~2.5H)!
# Get the frequency bands powers 
for mc, m_cond in enumerate(m_conds):
        path_in_eeg = path_eeg + m_cond + '/' + c_styles + '/'
        path_out_pwr = path_pwr + m_cond + '/' + c_styles + '/'
        path_out_pwr_roi = path_pwr_roi + m_cond + '/' + c_styles + '/'
        eeg_pow_extract(path_in_eeg, path_out_pwr, path_out_pwr_roi, mon_path, roi, fs, bands, freqs, len_win, overlap, hpf_freq, lpf_freq, mirror_len, mirror_break, cut)
        

------------------------------------------------------------------------------------------------------------------

In [ ]:
# Process data to extract frequency bands powers features 
# (e.g remove mirror padding, z-score, select HA and LA samples)
# Loop over the movement conditions:
for mc, m_cond in enumerate(m_conds):
    path_in_eeg_pow = path_pwr_roi + m_cond + '/' + c_styles + '/'
    path_in_ar = path_ar + m_cond + '/' + c_styles + '/'
    eeg_features[m_cond] = NVR_eeg_features(path_in_eeg_pow, path_in_ar, roi, cut)

In [ ]:
# Save eeg_features into file
filename_pwr = 'eeg_features.mat'
savemat(path_pwr_roi + filename_pwr, eeg_features)

# Prepare data for Linear Mixed Modeling (LMM) in R

In [ ]:
# Load eeg_features from file
eeg_features = loadmat(path_pwr_roi + 'eeg_features.mat', simplify_cells=True)

In [ ]:
# Create a design dataframe for the statistical analysis
design = []
for i, subj in enumerate(SJs_nomov):
    if subj not in SJs_mov:
        mov_conds = ['nomov']
    else:
        mov_conds = m_conds
    for m in mov_conds:
        if m == 'mov':
            sj_idx = SJs_mov.index(subj)
        else:
            sj_idx = i
        for a in a_conds:
            delta = castToList(eeg_features[m][sj_idx][a + '_delta_meanROI'])
            theta = castToList(eeg_features[m][sj_idx][a + '_theta_meanROI'])
            alpha = castToList(eeg_features[m][sj_idx][a + '_alpha_meanROI'])
            beta = castToList(eeg_features[m][sj_idx][a + '_beta_meanROI'])
            gamma = castToList(eeg_features[m][sj_idx][a + '_gamma_meanROI'])
            #
            z_delta = castToList(eeg_features[m][sj_idx][a + '_z_delta_meanROI'])
            z_theta = castToList(eeg_features[m][sj_idx][a + '_z_theta_meanROI'])
            z_alpha = castToList(eeg_features[m][sj_idx][a + '_z_alpha_meanROI'])
            z_beta = castToList(eeg_features[m][sj_idx][a + '_z_beta_meanROI'])
            z_gamma = castToList(eeg_features[m][sj_idx][a + '_z_gamma_meanROI'])
            for s, s_delta in enumerate(delta):
                design.append([subj, m, a,
                               delta[s], theta[s], alpha[s], beta[s], gamma[s], 
                               z_delta[s], z_theta[s], z_alpha[s], z_beta[s], z_gamma[s]
                               ])

In [ ]:
# Convert into a pandas dataframe
design_df = pd.DataFrame(design, columns=['SJ', 'mov_cond', 'arousal',
                                          'delta', 'theta', 'alpha', 'beta', 'gamma',
                                          'z_delta', 'z_theta', 'z_alpha', 'z_beta', 'z_gamma' 
                                          ])

In [ ]:
# Save the design dataframe into a csv file
filename = path_pwr_roi + 'nvr_roi_eeg_features.csv'
design_df.to_csv(filename, index=False, na_rep ='NaN')